In [1]:
# deep-text-recognition-benchmark의 dependency 설치
# !pip install lmdb pillow torchvision nltk natsort fire

### 2-1. [한국어 글자체 이미지](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=81)

In [2]:
import os
import glob
import json
import natsort
import pandas as pd

from tqdm.auto import tqdm, trange
from PIL import Image
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

/home/osh/.conda/envs/osh/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# train = pd.read_csv('/home/osh/Text Recognition/train.csv')[['img_path', 'label']]
# for i, e in enumerate(train['img_path']):
#     e = "/home/osh/Text Recognition" + e[1:]
#     train['img_path'][i] = e

# train

In [4]:

# train.to_csv("train2.csv", index = False, header = True)

In [5]:
# train = pd.read_csv('train2.csv')
# train

In [6]:
# test = pd.read_csv('test2.csv')
# test

In [7]:
# test = pd.read_csv('/home/osh/Text Recognition/test.csv')[['img_path']]
# for i, e in enumerate(test['img_path']):
#     e = "/home/osh/Text Recognition" + e[1:]
#     test['img_path'][i] = e
# test.to_csv('test2.csv', index = False)
# test

## 3. Dacon 데이터 전처리 및 lmdb 데이터 생성

### 3-1. train.csv 수정

In [8]:
# 라벨링이 잘못되어있거나 누락된 데이터 수정 -> train_edit.csv 로 저장
# train_csv_path = '/home/osh/Ocr/customocr-master/customocr-master/train2.csv'
# train_csv = pd.read_csv(train_csv_path)
# test_csv_path = '/home/osh/Ocr/customocr-master/customocr-master/test2.csv'
# test_csv = pd.read_csv(test_csv_path)

# # lmdb 데이터 생성을 위한 gt 파일 생성
# train_csv[:-1000].to_csv('./lmdb_gt/train.txt', sep='\t', header=False, index=False)

# # 최종 학습 단계의 모델 저장을 위한 임시 validation gt 파일 생성
# train_csv[-1000:].to_csv('./lmdb_gt/valid.txt', sep='\t', header=False, index=False)

In [9]:
# test_csv

### 3-2. 학습에 사용할 lmdb 데이터 생성

In [10]:
# '''
# window 환경에서 작업 시의 lmdb 파일 생성 명령어:
# !python ./hallymocr/create_lmdb_dataset.py --inputPath [데이터 root 경로] --gtFile [txt 파일 경로] --outputPath [데이터 저장 경로] --file_size [데이터 총 크기(GB)]

# linux 환경에서 작업 시의 lmdb 파일 생성 명령어:
# !python3 ./hallymocr/create_lmdb_dataset.py --inputPath [데이터 root 경로] --gtFile [txt 파일 경로] --outputPath [데이터 저장 경로] --file_size [데이터 총 크기(GB)]
# '''
# # !python ./hallymocr/create_lmdb_dataset.py --inputPath ./images/crop_valid --gtFile ./lmdb_data/hub_vlabel.txt --outputPath ./result/hvalid --file_size 5

# # print('한국어 글자체 (tiw) lmdb 생성')
# # !python ./hallymocr/create_lmdb_dataset.py --inputPath '' --gtFile ./lmdb_gt/crop_data.txt --outputPath ./result/tiw --file_size 10

# # print('야외 실제 촬영 데이터 lmdb 생성')
# # !python ./hallymocr/create_lmdb_dataset.py --inputPath '' --gtFile ./lmdb_gt/hub_tlabel.txt --outputPath ./result/htrain --file_size 20

# print('Dacon 제공 데이터셋 ( : -1000 만큼 train 할당')
# !python /home/osh/Ocr/customocr-master/customocr-master/hallymocr/create_lmdb_dataset.py --inputPath '' --gtFile /home/osh/Ocr/customocr-master/customocr-master/lmdb_gt/train.txt --outputPath /home/osh/Ocr/customocr-master/customocr-master/result/train --file_size 5

# print('Dacon 제공 데이터셋 ( -1000 :  만큼 valid 할당')
# !python /home/osh/Ocr/customocr-master/customocr-master/hallymocr/create_lmdb_dataset.py --inputPath '' --gtFile /home/osh/Ocr/customocr-master/customocr-master/lmdb_gt/valid.txt --outputPath /home/osh/Ocr/customocr-master/customocr-master/result/valid --file_size 2

# print('Dacon 제공 데이터셋 ( -1000 :  만큼 test 할당')
# !python /home/osh/Ocr/customocr-master/customocr-master/hallymocr/create_lmdb_dataset.py --inputPath '' --gtFile /home/osh/Ocr/customocr-master/customocr-master/lmdb_gt/test.txt --outputPath /home/osh/Ocr/customocr-master/customocr-master/result/test --file_size 5

## 4. OCR 모델 학습
1차 학습은 외부데이터로만 학습을 진행(crop약 160만장)하며, validation set은 Dacon측에서 제공한 train셋을 사용합니다.

In [11]:
ko_txt = ''
e = ord('가')

for i in range(11172):
    ko_txt += chr(e + i)

ko_txt += ' '

In [12]:
import sys
import random
import string
import torch.backends.cudnn as cudnn
import torch.utils.data
import numpy as np

sys.path.append("/home/osh/Ocr/customocr-master/customocr-master/hallymocr")
sys.path.append("/home/osh/Text Recognition/hallymocr")
# sys.path.insert(0, '/home/osh/Ocr/customocr-master/customocr-master/hallymocr')
from hallymocr.train import train
from hallymocr.test import test

In [13]:
# 1차 학습 하이퍼파라미터 설정
opt = {
    'exp_name': None,
    'train_data': '/home/osh/Ocr/customocr-master/customocr-master/result',
    'valid_data': '/home/osh/Ocr/customocr-master/customocr-master/result/valid',
    'manualSeed': 1111,
    'workers': 4,
    'batch_size': 32,
    'num_iter': 120000,
    'valInterval': 1000,
    'saved_model': '',

    'FT': False,
    'adam': False,
    'lr': 1,
    'beta1': 0.9,
    'rho': 0.95,
    'eps': 1e-8,
    'grad_clip': 5,
    'baiduCTC': False,
    'select_data': 'train',
    'batch_ratio': '1',
    'total_data_usage_ratio': '1',
    'batch_max_length': 15,

    'imgH': 64,
    'imgW': 224,
    'rgb': False,
    'character': ko_txt,
    'sensitive': False,
    'PAD': False,
    'data_filtering_off': False,
    'Transformation': 'TPS',  # None|TPS
    'FeatureExtraction': 'ResNet',  # VGG|ResNet|RCNN
    'SequenceModeling': 'BiLSTM',  # None|BiLSTM
    'Prediction': 'Attn',  # CTC|Attn
    'num_fiducial': 20,
    'input_channel': 1,
    'output_channel': 512,
    'hidden_size': 256,
}

# 모델 추가 세부사항 설정
if not opt['exp_name']:
    opt['exp_name'] = '{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}'.format(**opt)
    opt['exp_name'] += '-Seed{manualSeed}'.format(**opt)
    # print(opt.exp_name)

os.makedirs('./saved_models/{exp_name}'.format(**opt), exist_ok=True)

""" Seed and GPU setting """
random.seed(opt['manualSeed'])
np.random.seed(opt['manualSeed'])
torch.manual_seed(opt['manualSeed'])
torch.cuda.manual_seed(opt['manualSeed'])

cudnn.benchmark = True
cudnn.deterministic = True
opt['num_gpu'] = torch.cuda.device_count()

if opt['num_gpu'] > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    opt['workers'] = opt['workers'] * opt['num_gpu']
    opt['batch_size'] = opt['batch_size'] * opt['num_gpu']

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0


### 모델 훈련

In [14]:
# train
train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /home/osh/Ocr/customocr-master/customocr-master/result
opt.select_data: ['train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    /home/osh/Ocr/customocr-master/customocr-master/result	 dataset: train
sub-directory:	/train	 num samples: 75888
num total samples of train: 75888 x 1 (total_data_usage_ratio) = 75888
num samples of train per batch: 64 x 1.0 (batch_ratio) = 64
--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
dataset_root: /home/osh/Ocr/customocr-master/c

  0%|          | 1/120000 [00:10<352:44:22, 10.58s/it]

[1/120000] Train loss: 9.34474, Valid loss: 9.26409, Elapsed_time: 4.33333
Current_accuracy : 0.000, Current_norm_ED  : 0.00
Best_accuracy    : 0.000, Best_norm_ED     : 0.00
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
자네                        | 큕쓨쓨콀콀콀콀콀콀콀콀콀콀콀콀           | 0.0000	False
법칙                        | 큕댚댚씜씜씜씜씜씜혝혝혝혝혝혝           | 0.0000	False
겁                         | 큕댚댚씜씜씜씜씜씜씜씜씜혝혝혝           | 0.0000	False
알                         | 큕댚콀콀콀콀콀콀콀콀콀큕갺큕갺           | 0.0000	False
포스터                       | 큕몚쓨씜씜씜씜씜씜씜씜씜씜씜씜           | 0.0000	False
--------------------------------------------------------------------------------


  1%|          | 1000/120000 [06:52<62:09:17,  1.88s/it]

[1000/120000] Train loss: 4.50635, Valid loss: 4.10225, Elapsed_time: 407.67607
Current_accuracy : 0.000, Current_norm_ED  : 0.07
Best_accuracy    : 0.000, Best_norm_ED     : 0.07
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
돌아다니다                     | 아다하다                      | 0.0002	False
종                         |                           | 0.0000	False
원                         |                           | 0.0000	False
과장                        | 그                         | 0.0128	False
분량                        | 아다                        | 0.0010	False
--------------------------------------------------------------------------------


...

[13000/120000] Train loss: 1.10147, Valid loss: 1.03250, Elapsed_time: 5599.19059
Current_accuracy : 52.000, Current_norm_ED  : 0.65
Best_accuracy    : 52.000, Best_norm_ED     : 0.65
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
늦다                        | 붓다                        | 0.0625	False
단위                        | 단위                        | 0.7182	True
제작하다                      | 계작하다                      | 0.1541	False
기억나다                      | 기억나다                      | 0.2644	True
자기                        | 자기                        | 0.9773	True
--------------------------------------------------------------------------------


 12%|█▏        | 14000/120000 [1:40:33<67:03:31,  2.28s/it]

[14000/120000] Train loss: 0.98997, Valid loss: 0.98442, Elapsed_time: 6027.37309
Current_accuracy : 55.500, Current_norm_ED  : 0.68
Best_accuracy    : 55.500, Best_norm_ED     : 0.68
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
그리로                       | 그리로                       | 0.7085	True
새끼                        | 새끼                        | 0.6153	True
보도                        | 보도                        | 0.9444	True
켜다                        | 켜다                        | 0.0556	True
와                         | 와                         | 0.3768	True
--------------------------------------------------------------------------------


 12%|█▏        | 14017/120000 [1:40:41<12:41:18,  2.32it/s]


KeyboardInterrupt: 

### Dacon 데이터로 2차 학습
외부데이터로 학습된 1차 학습에서 가장 좋은 정확도를 보인 모델을 불러와 </br>
오타, 띄어쓰기, 이상값 등을 수작업으로 정제한 Dacon Train파일을 학습데이터로 사용하여 추가학습을 진행합니다. </br>
이때 train셋은 Dacon Train셋의 -1000번째 까지이며 validation은 Dacon Train셋의 끝에서 1000번째 까지의 데이터를 사용합니다.

In [ ]:
opt['exp_name'] = '{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}'.format(**opt)
opt['saved_model'] = 'saved_models/{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}/best_accuracy.pth'.format(**opt)
opt['num_iter'] = 1000
opt['lr'] = 0.001
opt['select_data'] = 'train'
opt['valid_data'] = './result/valid'
opt['batch_ratio'] = '1'

In [ ]:
train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /home/osh/Ocr/customocr-master/customocr-master/result
opt.select_data: ['train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    /home/osh/Ocr/customocr-master/customocr-master/result	 dataset: train
sub-directory:	/train	 num samples: 75888
num total samples of train: 75888 x 1 (total_data_usage_ratio) = 75888
num samples of train per batch: 64 x 1.0 (batch_ratio) = 64
--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
dataset_root: /home/osh/Ocr/customocr-master/c

  0%|          | 1/1000 [00:06<1:43:51,  6.24s/it]

[1/1000] Train loss: 0.02943, Valid loss: 0.04565, Elapsed_time: 0.43189
Current_accuracy : 97.900, Current_norm_ED  : 0.99
Best_accuracy    : 97.900, Best_norm_ED     : 0.99
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
계산하다                      | 계산하다                      | 0.9990	True
눈                         | 눈                         | 0.9991	True
내다보다                      | 내다보다                      | 0.9997	True
그래                        | 그래                        | 0.9998	True
믓                         | 믓                         | 0.9730	True
--------------------------------------------------------------------------------


100%|█████████▉| 999/1000 [07:00<00:00,  2.37it/s]

[1000/1000] Train loss: 0.00447, Valid loss: 0.05232, Elapsed_time: 416.97926
Current_accuracy : 97.800, Current_norm_ED  : 0.99
Best_accuracy    : 97.900, Best_norm_ED     : 0.99
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
켱                         | 켱                         | 0.9868	True
알                         | 알                         | 0.9975	True
고전                        | 고전                        | 0.9998	True
낯설다                       | 낯설다                       | 0.9964	True
아뇨                        | 아뇨                        | 0.9986	True
--------------------------------------------------------------------------------
end the training


## 5. 최종 결과

### 모델 예측

In [ ]:
# # 저장된 모델 load
# opt['saved_model'] = 'saved_models/{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}/best_accuracy.pth'.format(**opt)
# opt['test_data'] = '/home/osh/Ocr/customocr-master/customocr-master/result/test'
# # opt['test_data'] = './test'


In [ ]:
# 저장된 모델 load
opt['saved_model'] = 'saved_models/{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}/best_accuracy.pth'.format(**opt)
opt['test_data'] = "/home/osh/Text Recognition/test"

In [ ]:
result = test(opt)

model input parameters 64 224 20 1 512 256 11175 15 TPS ResNet BiLSTM Attn
loading pretrained model from saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth


### CSV 파일 생성

In [ ]:
# result_csv_path = '/home/osh/Text Recognition/sample_submission.csv'
result_csv_path = '/home/osh/Text Recognition/sample_submission.csv'
result_csv = pd.read_csv(result_csv_path)

result_csv['label'] = result

In [ ]:
from datetime import datetime

time = datetime.now().strftime('%m%d_%H%M')
csv_name = f'{time}.csv'
result_csv.to_csv(csv_name, index=False, encoding='utf8')

In [ ]:
pd.read_csv(csv_name)

,id,label
0,TEST_00000,날말
1,TEST_00001,상황
2,TEST_00002,받아들이다
3,TEST_00003,바구니
4,TEST_00004,살
...,...,...
74116,TEST_74116,캐나다
74117,TEST_74117,사무
74118,TEST_74118,친절하다
74119,TEST_74119,쪽
